In [10]:
from google.colab import drive
drive.mount('/content/drive')
zip_file_path = "/content/drive/MyDrive/archi.zip"

extract_dir = "/content/extracted_leapGestRecog"
import zipfile
import os

# Check if the path exists
if os.path.exists(zip_file_path):
    print(f"The path {zip_file_path} exists.")
    # Check if it's a file
    if os.path.isfile(zip_file_path):
        print(f"{zip_file_path} is a file.")
        # Try to open it as a zip file
        try:
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_dir)
                print(f"Successfully extracted to {extract_dir}")
        except zipfile.BadZipFile:
            print(f"{zip_file_path} is not a valid zip file.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
else:
    print(f"The path {zip_file_path} does not exist.")
import os

def print_directory_structure(root_dir):
    for root, dirs, files in os.walk(root_dir):
        level = root.replace(root_dir, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f"{indent}{os.path.basename(root)}/")
        sub_indent = ' ' * 4 * (level + 1)
        for f in files:
            print(f"{sub_indent}{f}")

# Verify the structure of the extracted directory
print_directory_structure("/content/extracted_leapGestRecog")












Streaming output truncated to the last 5000 lines.
                frame_07_07_0198.png
                frame_07_07_0120.png
                frame_07_07_0009.png
                frame_07_07_0048.png
                frame_07_07_0100.png
                frame_07_07_0134.png
                frame_07_07_0033.png
                frame_07_07_0193.png
                frame_07_07_0027.png
                frame_07_07_0095.png
                frame_07_07_0181.png
                frame_07_07_0022.png
                frame_07_07_0177.png
                frame_07_07_0081.png
                frame_07_07_0023.png
                frame_07_07_0165.png
                frame_07_07_0176.png
                frame_07_07_0174.png
                frame_07_07_0090.png
                frame_07_07_0062.png
                frame_07_07_0130.png
                frame_07_07_0109.png
                frame_07_07_0088.png
                frame_07_07_0015.png
                frame_07_07_0092.png
                frame_07

In [11]:
import cv2
import numpy as np

imagepaths = []
X = []  # Image data
y = []  # Labels

# Walk through the directory structure to load images and labels
for root, dirs, files in os.walk("/content/extracted_leapGestRecog"):
    for file in files:
        if file.endswith(".png"):  # Adjust the extension if needed
            imagepaths.append(os.path.join(root, file))
            img = cv2.imread(os.path.join(root, file))  # Reads image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            img = cv2.resize(img, (320, 120))  # Resize image
            X.append(img)

            # Processing label from the folder name
            category = os.path.basename(os.path.dirname(root))
            label = int(category.split("_")[0][1])  # Adjusted to handle label extraction correctly
            y.append(label)

# Convert lists to numpy arrays
X = np.array(X, dtype="uint8")
X = X.reshape(len(imagepaths), 120, 320, 1)  # Reshape for CNN
y = np.array(y)

print("Images loaded:", len(X))
print("Labels loaded:", len(y))


Images loaded: 20000
Labels loaded: 20000


In [12]:
# Percentage of images that we want to use for testing.
# The rest is used for training.
from sklearn.model_selection import train_test_split
ts = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=42)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(120, 320, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Configures the model for training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=2, validation_data=(X_test, y_test))


Epoch 1/5
219/219 - 16s - loss: 0.7601 - accuracy: 0.9221 - val_loss: 0.0173 - val_accuracy: 0.9927 - 16s/epoch - 72ms/step
Epoch 2/5
219/219 - 14s - loss: 0.0201 - accuracy: 0.9914 - val_loss: 0.0165 - val_accuracy: 0.9923 - 14s/epoch - 63ms/step
Epoch 3/5
219/219 - 12s - loss: 0.0316 - accuracy: 0.9877 - val_loss: 0.0347 - val_accuracy: 0.9847 - 12s/epoch - 56ms/step
Epoch 4/5
219/219 - 12s - loss: 0.0186 - accuracy: 0.9917 - val_loss: 0.0120 - val_accuracy: 0.9915 - 12s/epoch - 56ms/step
Epoch 5/5
219/219 - 12s - loss: 0.0136 - accuracy: 0.9921 - val_loss: 0.0118 - val_accuracy: 0.9925 - 12s/epoch - 56ms/step


In [16]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy: {:2.2f}%'.format(test_acc*100))
predictions = model.predict(X_test) # Make predictions towards the test set
y_pred = np.argmax(predictions, axis=1) # Transform predictions into 1-D array with label number
# H = Horizontal
import pandas as pd
# V = Vertical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
pd.DataFrame(confusion_matrix(y_test, y_pred),
             columns=["Predicted Thumb Down", "Predicted Palm (H)", "Predicted L", "Predicted Fist (H)", "Predicted Fist (V)", "Predicted Thumbs up", "Predicted Index", "Predicted OK", "Predicted Palm (V)", "Predicted C"],
             index=["Actual Thumb Down", "Actual Palm (H)", "Actual L", "Actual Fist (H)", "Actual Fist (V)", "Actual Thumbs up", "Actual Index", "Actual OK", "Actual Palm (V)", "Actual C"])


188/188 [==============================] - 2s 9ms/step - loss: 0.0118 - accuracy: 0.9925
Test accuracy: 99.25%
188/188 [==============================] - 1s 7ms/step


,Predicted Thumb Down,Predicted Palm (H),Predicted L,Predicted Fist (H),Predicted Fist (V),Predicted Thumbs up,Predicted Index,Predicted OK,Predicted Palm (V),Predicted C
Actual Thumb Down,610,0,0,0,0,0,0,0,0,0
Actual Palm (H),0,590,0,0,0,0,0,0,0,0
Actual L,0,0,613,0,0,15,0,0,0,0
Actual Fist (H),0,0,0,624,0,0,0,0,0,0
Actual Fist (V),0,0,0,0,583,0,0,0,0,0
Actual Thumbs up,0,0,30,0,0,554,0,0,0,0
Actual Index,0,0,0,0,0,0,580,0,0,0
Actual OK,0,0,0,0,0,0,0,618,0,0
Actual Palm (V),0,0,0,0,0,0,0,0,618,0
Actual C,0,0,0,0,0,0,0,0,0,565
